In [18]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [23]:
df = pd.read_csv("clean_pset1.csv")

################
#5.a
################

#Logit to estimate propensity score
y_log_reg = df['tobacco']
#outcome
y = ['dbrwt']
#treatment
D = ['tobacco']
#cor with y and D
x1 = ['alcohol', 'mrace3_2', 'mrace3_3', 'ormothhis', 'adeq_2.0', 'adeq_3.0', 'cardiac', 'pre4000', 'phyper', 'chyper', 
     'diabetes', 'anemia', 'lung', 'dlivord', 'educ_0.0', 'educ_1.0', 'educ_2.0', 'dgestat', 'dmage', 'dmar']
#cor with D not y
x2 = []
#cor with y not D
x3 = ['csex', 'tot_2.0','tot_3.0','tot_4.0','tot_5.0','tot_6.0','tot_7.0','tot_8.0', 'live_1.0', 'live_2.0', 
      'live_3.0','live_4.0','live_5.0','live_6.0','live_7.0','live_8.0','live_9.0', 
    'cnto_1.0', 'cnto_2.0', 'cnto_3.0', 'plur_1']

X_log_reg = df[x1+x3]

model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_log_reg, y_log_reg)

#2nd column gives us predictions
predictions = model.predict_proba(X_log_reg)[:,1]
df['predictions'] = model.predict_proba(X_log_reg)[:,1]


#Calculate weights
wt = (df['tobacco'] / predictions) + (1 - df['tobacco']) /(1 -  predictions)

#Demeaned X matrix

for item in x1+x3:
    df[item+'demeaned'] = df[item].sub(df[item].mean())
    
#multiply by D
for item in x1+x3:    
    df['tobacco*'+item] = df[item+'demeaned']*df['tobacco']


#concatenate
d_times_demeaned_X = df[['tobacco*alcohol',
       'tobacco*mrace3_2', 'tobacco*mrace3_3', 'tobacco*ormothhis',
       'tobacco*adeq_2.0', 'tobacco*adeq_3.0', 'tobacco*cardiac',
       'tobacco*pre4000', 'tobacco*phyper', 'tobacco*chyper',
       'tobacco*diabetes', 'tobacco*anemia', 'tobacco*lung', 'tobacco*dlivord',
       'tobacco*educ_0.0', 'tobacco*educ_1.0', 'tobacco*educ_2.0',
       'tobacco*dgestat', 'tobacco*dmage', 'tobacco*dmar', 'tobacco*csex',
       'tobacco*tot_2.0', 'tobacco*tot_3.0', 'tobacco*tot_4.0',
       'tobacco*tot_5.0', 'tobacco*tot_6.0', 'tobacco*tot_7.0',
       'tobacco*tot_8.0', 'tobacco*live_1.0', 'tobacco*live_2.0',
       'tobacco*live_3.0', 'tobacco*live_4.0', 'tobacco*live_5.0',
       'tobacco*live_6.0', 'tobacco*live_7.0', 'tobacco*live_8.0',
       'tobacco*live_9.0', 'tobacco*cnto_1.0', 'tobacco*cnto_2.0',
       'tobacco*cnto_3.0', 'tobacco*plur_1']]

#Outcome: birthweight
y_log_reg = df['dbrwt']

#Create final covariates matrix: 
double_robust_reg_X = pd.concat([df['tobacco'],
                                 X_log_reg,
                                 d_times_demeaned_X], axis=1)


fit_wls = sm.WLS(y_log_reg, double_robust_reg_X, weights=wt).fit()
print(fit_wls.summary())



                                 WLS Regression Results                                
Dep. Variable:                  dbrwt   R-squared (uncentered):                   0.980
Model:                            WLS   Adj. R-squared (uncentered):              0.980
Method:                 Least Squares   F-statistic:                          7.012e+04
Date:                Sat, 30 Sep 2023   Prob (F-statistic):                        0.00
Time:                        16:37:16   Log-Likelihood:                     -8.8557e+05
No. Observations:              114610   AIC:                                  1.771e+06
Df Residuals:                  114528   BIC:                                  1.772e+06
Df Model:                          82                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------